In [ ]:
! pip install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

! mkdir mri_dataset

! unzip brain-tumor-mri-dataset.zip -d mri_dataset

! kaggle datasets download -d rahimanshu/figshare-brain-tumor-classification
! mkdir glioma_dataset
! unzip figshare-brain-tumor-classification.zip -d glioma_dataset

In [2]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sklearn.metrics as skm 
import seaborn as sns

from tqdm import tqdm
import os
from sklearn.utils import shuffle, class_weight
import cv2

from random import randint

from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils.vis_utils import plot_model

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

%config Completer.use_jedi = False

import warnings
warnings.simplefilter("ignore")

In [3]:
np.random.RandomState(10)
tf.random.set_seed(10)

In [ ]:
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

X_treino = []
Y_treino = []

X_teste = []
Y_teste = []

image_size = 128

glioma_folder_path = os.path.join('/content/glioma_dataset', 'glioma')

glioma_images = []
for j in tqdm(os.listdir(glioma_folder_path)):
    img = cv2.imread(os.path.join(glioma_folder_path,j))
    img = cv2.resize(img,(image_size, image_size))
    glioma_images.append(img)

glioma_labels = ['glioma'] * len(glioma_images)

glioma_images_train = glioma_images[:1126]
glioma_images_test = glioma_images[1126:1426]

glioma_labels_train = glioma_labels[:1126]
glioma_labels_test = glioma_labels[1126:1426]

X_treino.extend(glioma_images_train)
Y_treino.extend(glioma_labels_train)
X_teste.extend(glioma_images_test)
Y_teste.extend(glioma_labels_test)

for i in class_names:
    if i != 'glioma':
        folderPath = os.path.join('/content/mri_dataset','Training',i)
        for j in tqdm(os.listdir(folderPath)):
            img = cv2.imread(os.path.join(folderPath,j))
            img = cv2.resize(img,(image_size, image_size))
            X_treino.append(img)
            Y_treino.append(i)

        folderPath = os.path.join('/content/mri_dataset','Testing',i)
        for j in tqdm(os.listdir(folderPath)):
            img = cv2.imread(os.path.join(folderPath,j))
            img = cv2.resize(img,(image_size, image_size))
            X_teste.append(img)
            Y_teste.append(i)

In [5]:
X_treino = np.array(X_treino)
Y_treino = np.array(Y_treino)

X_teste = np.array(X_teste)
Y_teste = np.array(Y_teste)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(Y_treino),
                                                 y = Y_treino)

weight = {i : class_weights[i] for i in range(4)}
    
print(weight)

In [7]:
Y_treino = np.unique(Y_treino, return_inverse = True)[1]
Y_teste = np.unique(Y_teste, return_inverse = True)[1]

In [ ]:
Y_treino_count = np.bincount(Y_treino)

total = np.sum([Y_treino_count], axis=1)[0]

print(
    'Examples:\n    Total: {}\n    Glioma: {} ({:.2f}% of total)\n    Meningioma: {} ({:.2f}% of total)\n    No Tumor: {} ({:.2f}% of total)\n    Pituitary: {} ({:.2f}% of total)'.format(
    total, 
    Y_treino_count[0], 100 * Y_treino_count[0] / total, 
    Y_treino_count[1], 100 * Y_treino_count[1] / total, 
    Y_treino_count[2], 100 * Y_treino_count[2] / total, 
    Y_treino_count[3], 100 * Y_treino_count[3] / total, ))

In [9]:
X_treino = X_treino/255.0
X_teste = X_teste/255.0

In [ ]:
plt.figure(figsize=(20,20))
sum = 1017
for i in range(4):
    plt.subplot(5,5,i+1)
    plt.imshow(X_treino[i + sum])
    plt.xlabel(class_names[Y_treino[i + sum]], fontsize=24)

    sum = sum + 1350
plt.show()

In [11]:
def get_model():

    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', 
                    input_shape=(image_size,image_size,3), padding='same'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.20))

    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.30))

    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))             
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.40))

    model.add(Flatten())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.60))

    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
    
    return model

In [12]:
#! pip install visualkeras

#import visualkeras
#from PIL import ImageFont

model = get_model()

#font = ImageFont.truetype("Roboto-Regular.ttf", 24)
#visualkeras.layered_view(model, legend=True, font=font)

#model.summary()
#plot_model(model, to_file='model_plot.png', show_shapes = True, 
#           show_layer_names = False, show_layer_activations = True)

In [13]:
erlstp = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=18)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=5)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights/weights.hdf5',
    verbose=1,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [14]:
def fit_and_evaluate(t_x, val_x, t_y, val_y, EPOCHS, BATCH_SIZE, class_w):
    model = None
    model = get_model()
    results = model.fit(t_x, t_y, epochs=EPOCHS, batch_size=BATCH_SIZE,
                validation_data = (val_x, val_y),
                class_weight=class_w,
                callbacks=[erlstp, reduce_lr, model_checkpoint], verbose=1)  
    
    print("\nValidation Score: ", model.evaluate(val_x, val_y))
    return results

In [ ]:
n_folds=5
epochs=100
batch_size=32

model_history = [] 

for i in range(n_folds):
    print("Training on Fold: ", i + 1)

    X_t, X_v, label_t, label_v = train_test_split(
      X_treino, Y_treino, test_size = 0.2, random_state = 13
    )

    X_t, label_t = shuffle(X_t, label_t, random_state = 13)
                                               
    model_history.append(fit_and_evaluate(X_t, X_v, label_t, label_v, epochs, batch_size, weight))
    print("=======" * 12, end="\n\n\n")

In [ ]:
colors = ['black', 'red', 'green', 'purple', 'gray']
labels = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']

plt.figure(figsize=(10,7))
plt.title('Train Loss vs Val Loss')
for i, color in enumerate(colors):
    plt.plot(model_history[i].history['loss'], label='Train Loss '+labels[i], color=color)
    plt.plot(model_history[i].history['val_loss'], label='Val Loss '+labels[i], color=color, linestyle = "dashdot")
plt.legend()
plt.show()

plt.figure(figsize=(10,7))
plt.title('Train Accuracy vs Val Accuracy')
for i, color in enumerate(colors):
    plt.plot(model_history[i].history['accuracy'], label='Train Accuracy '+labels[i], color=color)
    plt.plot(model_history[i].history['val_accuracy'], label='Val Accuracy '+labels[i], color=color, linestyle = "dashdot")
plt.legend()
plt.show()


In [ ]:

#Load the model that was saved by ModelCheckpoint
model = load_model('weights/weights.hdf5')

results = model.evaluate(X_teste, Y_teste)
print("Loss: {:0.4f}".format(results[0]))

In [ ]:
# Testando o modelo com o conjunto de teste, separado anteriormente

predicted_classes = model.predict(X_teste)
predicted_classes = np.argmax(np.round(predicted_classes), axis = 1)

print(classification_report(Y_teste, predicted_classes, target_names = class_names))

In [ ]:
# Gerando e apresentando a matriz de confusão

cmat = confusion_matrix(Y_teste, predicted_classes)
cm_df = pd.DataFrame(cmat) 

cmat_df = pd.DataFrame(cmat,
                     index = class_names, 
                     columns = class_names)

plt.figure(figsize=(10,6))

sns.heatmap(cmat, annot = True, cmap = 'Blues', fmt = "d",
            cbar = False,
            xticklabels = class_names,
            yticklabels = class_names)

plt.title('Matriz de Confusão')
plt.show()

In [29]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
img = cv2.imread('/content/mri_dataset/Testing/meningioma/Te-me_0025.jpg')
orig_img = img
img = cv2.resize(img, (image_size, image_size))

img = img.astype('float32') / 255.0

x = np.array(img)
x = np.expand_dims(x, axis=0)

layer_name = 'conv2d_6'

heatmap = make_gradcam_heatmap(x, model, layer_name)

orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB)

heatmap_resized = cv2.resize(heatmap_colored, (orig_img.shape[1], orig_img.shape[0]))

superimposed_img = np.uint8(heatmap_resized * 0.5 + orig_img)

plt.imshow(superimposed_img)
plt.show()